!!! 
Notebook déplacé, les chemins sont à revoir.

Autrement, ce traitement n'existe que pour combler les erreurs de concat.ipynb. 

In [1]:
import duckdb
import pandas as pd 
import polars as pl 
import pyarrow as pa
import pyarrow.parquet as pq

In [18]:
# Schema 

new_chem = pa.schema([
        ('Id_Fichier', pa.int32()),
        ('DEP', pa.string()), #Corse 
        ('REG', pa.int32()),
        ('EPCI', pa.int32()),
        ('CODGEO', pa.int32()), 
        ('LIBGEO', pa.string()),
        ('denominationUniteLegale', pa.string()),
        ('libelleCategorieJuridique', pa.string()), 
        ('DteStr', pa.date32()), 
        ('LibellePoste', pa.string()), 
        ('TypOpBudg', pa.int16()), #1 ou 2
        ('NatCEPL', pa.int32()), 
        ('NatDec', pa.string()),  
        ('Exer', pa.int32()), 
        ('ProjetBudget', pa.bool_()), 
        ('NatVote', pa.string()), 
        ('OpeEquip', pa.int32()), 
        ('VoteFormelChap', pa.bool_()), 
        ('TypProv', pa.int16()), 
        ('BudgPrec', pa.int16()), 
        ('ReprRes', pa.int16()), 
        ('NatFonc', pa.int16()),
        ('CodTypBud', pa.int16()),
        ('LibelleEtabPal', pa.string()),
        ('IdEtabPal', pa.int32()),
        ('LibelleEtab', pa.string()),
        ('IdEtab', pa.int32()),
        ('CodColl', pa.string()),
        ('CodInseeColl', pa.int32()),  
        ('CodBud', pa.string()),
        ('Nomenclature', pa.string()),
        ('Nature', pa.string()),
        ('Nature_Lib', pa.string()),
        ('Fonction', pa.string()),
        ('Fonction_Lib', pa.string()),
        ('ContNat', pa.string()),
        ('ContNat_Lib', pa.string()),
        ('LibCpte', pa.string()),
        ('ArtSpe', pa.bool_()),
        ('ContFon', pa.string()),
        ('CodRD', pa.string()),
        ('MtBudgPrec', pa.float32()),
        ('MtRARPrec', pa.float32()),
        ('MtPropNouv', pa.float32()),
        ('MtPrev', pa.float32()),
        ('CredOuv', pa.float32()),
        ('MtReal', pa.float32()),
        ('MtRAR3112', pa.float32()),
        ('siret', pa.string()),
        ('IdPost', pa.string())
    ])

In [19]:
#aa = pq.read_table('./haaaa.parquet', use_threads= True, memory_map= True)
aa = pl.scan_parquet('./haaaa.parquet').collect(streaming = True)
aa

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_transco = pd.read_parquet('./ETL_Lignes_Budget_EV/transcodage/parquet/transcodage_2020')
transco = df_transco[['Code', 'Lib_court', 'Libelle', 'Categorie', 'Nomenclature']]

In [ ]:
transco1 = transco[['Code', 'Lib_court', 'Categorie', 'Nomenclature']]

df_contnat = transco1[transco1['Categorie'] == 'Nature'][['Code', 'Lib_court', 'Nomenclature']].rename(columns = {
    'Code': 'ContNat', 'Lib_court' : 'ContNat_Lib'})

df_nature = transco1[transco1['Categorie'] == 'Nature_compte'][['Code', 'Lib_court', 'Nomenclature']].rename(columns = {
    'Code': 'Nature', 'Lib_court' : 'Nature_Lib'})

df_fonction = transco1[transco1['Categorie'] == 'Fonction_Ref'][['Code', 'Lib_court', 'Nomenclature']].rename(columns = {
    'Code': 'Fonction', 'Lib_court' : 'Fonction_Lib'})
df_contnat.head(1)

,ContNat,ContNat_Lib,Nomenclature
0,002,Résultat de fonctionnement reporté,M14-M14_COM_500_3500


In [6]:
conduck = duckdb.connect(database=':memory:')


def requetage(source = 'source', requete = 'demande'):
 conduck = duckdb.connect(database=':memory:')
 requete_donnees = requete 
 #conduck.sql(requete).write_parquet('./haaaa.parquet')
 pl_resultat = conduck.execute(requete_donnees).df()
 return pl_resultat


In [36]:

source_duck = conduck.read_parquet('./ETL_Lignes_Budget_EV/exploration_viz/db_2020.parquet')
source_siren = conduck.read_parquet('../projets/data/infos_siren_siret_cog_collectivites.parquet')

rts = requetage(source = source_duck, requete = 
'''SELECT DISTINCT BudgetHorsRAR FROM source ''')
rts

,BudgetHorsRAR
0,13000.00
1,2000.00
2,116500.00
3,41000
4,2940000.00
...,...
617232,28024.72
617233,4512.97
617234,1725.20
617235,6181.61


In [ ]:

d_nature = conduck.register('df_nature', df_nature)
d_contnat = conduck.register('df_contnat', df_contnat)
d_fonction = conduck.register('df_fonction', df_fonction)

NameError: name 'source_duck' is not defined

In [ ]:

source_duck = conduck.read_parquet('./ETL_Lignes_Budget_EV/exploration_viz/db_2020.parquet')
source_siren = conduck.read_parquet('../projets/data/infos_siren_siret_cog_collectivites.parquet')
d_nature = conduck.register('df_nature', df_nature)
d_contnat = conduck.register('df_contnat', df_contnat)
d_fonction = conduck.register('df_fonction', df_fonction)

In [15]:
tt1 = requetage(source = source_duck,
    requete='''
    SELECT 
       a.Id_Fichier, b.DEP, b.REG, b.EPCI, b.CODGEO, 
       b.LIBGEO, b.denominationUniteLegale, b.libelleCategorieJuridique,
       a.DteStr, a.LibellePoste, a.TypOpBudg,
       a.OpeCpteTiers, a.NatCEPL, a.NatDec,  
       
       CAST(a.Exer as INT) as Exer,
       a.ProjetBudget, a.NatVote, a.OpeEquip, a.VoteFormelChap, a.TypProv,
       a.BudgPrec, a.ReprRes, a.NatFonc, a.CodTypBud, a.LibelleEtabPal,
       a.IdEtabPal, a.LibelleEtab, a.IdEtab, a.CodColl, a.CodInseeColl,
       a.CodBud, a.Nomenclature, 
       
       a.Nature, df_nature.Nature_Lib,
       a.Fonction, df_fonction.Fonction_Lib,
       a.ContNat, df_contnat.ContNat_Lib,
       
       a.LibCpte, 
       a.ArtSpe, a.ContFon, a.CodRD, a.MtBudgPrec, a.MtRARPrec, 
       ROUND(CAST(a.MtPropNouv as FLOAT), 2) AS MtPropNouv, 
       ROUND(a.MtPrev, 2) AS MtPrev, 
       ROUND(a.CredOuv, 2) AS CredOuv, 
       ROUND(a.MtReal,2) AS MtReal, 
       ROUND(a.MtRAR3112, 2) AS MtRAR3112, a.OpBudg, 
       a.IdColl as siret, a.IdPost
       
    FROM source as a 
    LEFT JOIN source_siren as b
        ON a.IdColl = b.siret
    LEFT JOIN df_nature
        ON a.Nature = df_nature.Nature
    LEFT JOIN df_fonction
        ON a.Fonction = df_fonction.Fonction
    LEFT JOIN df_contnat
        ON a.ContNat = df_contnat.ContNat

    WHERE a.NatDec = '09'
        AND a.OpBudg = '0' 
''')

15min = 2.2g 

2020 ==> 12 045 929 lignes 

In [13]:
df1 = df[df.duplicated(subset=(df.columns))]
df1.shape 

: 

: 

In [12]:
df1 

,Id_Fichier,DEP,REG,EPCI,CODGEO,LIBGEO,denominationUniteLegale,libelleCategorieJuridique,DteStr,LibellePoste,TypOpBudg,OpeCpteTiers,NatCEPL,NatDec,Exer,ProjetBudget,NatVote,OpeEquip,VoteFormelChap,TypProv,BudgPrec,ReprRes,NatFonc,CodTypBud,LibelleEtabPal,IdEtabPal,LibelleEtab,IdEtab,CodColl,CodInseeColl,CodBud,Nomenclature,Nature,Nature_Lib,Fonction,Fonction_Lib,ContNat,ContNat_Lib,LibCpte,ArtSpe,ContFon,CodRD,MtBudgPrec,MtRARPrec,MtPropNouv,MtPrev,CredOuv,MtReal,MtRAR3112,OpBudg,siret,IdPost
742,697065,77,11.0,247700594,77243,Lagny-sur-Marne,COMMUNE LAGNY SUR MARNE,Commune et commune nouvelle,2021-03-16,CHELLES,nan,None,Commune de LAGNY-SUR-MARNE,09,2020,None,FcIc,1,1,1,2,3,3,P,None,None,VILLE DE LAGNY-SUR-MARNE,21770243000018,101,77243,40,M14-M14_COM_SUP3500,6232,Fêtes et cérémonies,024,Fêtes et cérémonies,011,Charges à caractère général,nan,None,None,D,NaN,NaN,NaN,NaN,92451.296875,77598.742188,NaN,0,21770243000018,077203
743,697065,77,11.0,247700594,77243,Lagny-sur-Marne,COMMUNE LAGNY SUR MARNE,Commune et commune nouvelle,2021-03-16,CHELLES,nan,None,Commune de LAGNY-SUR-MARNE,09,2020,None,FcIc,1,1,1,2,3,3,P,None,None,VILLE DE LAGNY-SUR-MARNE,21770243000018,101,77243,40,M14-M14_COM_SUP3500,6232,Fêtes et cérémonies,211,Ecoles maternelles,011,Charges à caractère général,nan,None,None,D,NaN,NaN,NaN,NaN,17590.000000,15917.250000,NaN,0,21770243000018,077203
744,697065,77,11.0,247700594,77243,Lagny-sur-Marne,COMMUNE LAGNY SUR MARNE,Commune et commune nouvelle,2021-03-16,CHELLES,nan,None,Commune de LAGNY-SUR-MARNE,09,2020,None,FcIc,1,1,1,2,3,3,P,None,None,VILLE DE LAGNY-SUR-MARNE,21770243000018,101,77243,40,M14-M14_COM_SUP3500,6232,Fêtes et cérémonies,415,Manifestations sportives,011,Charges à caractère général,nan,None,None,D,NaN,NaN,NaN,NaN,804.000000,803.700012,NaN,0,21770243000018,077203
745,697065,77,11.0,247700594,77243,Lagny-sur-Marne,COMMUNE LAGNY SUR MARNE,Commune et commune nouvelle,2021-03-16,CHELLES,nan,None,Commune de LAGNY-SUR-MARNE,09,2020,None,FcIc,1,1,1,2,3,3,P,None,None,VILLE DE LAGNY-SUR-MARNE,21770243000018,101,77243,40,M14-M14_COM_SUP3500,6232,Fêtes et cérémonies,64,Crèches et garderies,011,Charges à caractère général,nan,None,None,D,NaN,NaN,NaN,NaN,4400.000000,753.409973,NaN,0,21770243000018,077203
746,697065,77,11.0,247700594,77243,Lagny-sur-Marne,COMMUNE LAGNY SUR MARNE,Commune et commune nouvelle,2021-03-16,CHELLES,nan,None,Commune de LAGNY-SUR-MARNE,09,2020,None,FcIc,1,1,1,2,3,3,P,None,None,VILLE DE LAGNY-SUR-MARNE,21770243000018,101,77243,40,M14-M14_COM_SUP3500,6233,Foires et expositions,322,Musées,011,Charges à caractère général,nan,None,None,D,NaN,NaN,NaN,NaN,1536.030029,700.000000,NaN,0,21770243000018,077203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12999995,757749,79,75.0,200041317,79191,Niort,DEPARTEMENT DES DEUX SEVRES,Département,2021-03-01,Payeur Départemental,nan,nan,Départements/Ets Public Local/Ets Public Santé,09,2020,None,FcIc,true,false,1,1,1,3,P,None,22790001600352,Budget principal,22790001600352,011,None,00,M52-M52,6331,Versement de transport,80,Services communs,012,"Charges de personnel, frais assimilés",nan,None,None,D,0.0,0.0,0.0,0.0,1000.000000,418.000000,0.0,0,22790001600352,079090
12999996,757749,79,75.0,200041317,79191,Niort,DEPARTEMENT DES DEUX SEVRES,Département,2021-03-01,Payeur Départemental,nan,nan,Départements/Ets Public Local/Ets Public Santé,09,2020,None,FcIc,true,false,1,1,1,3,P,None,22790001600352,Budget principal,22790001600352,011,None,00,M52-M52,6331,Versement de transport,928,Autres,012,"Charges de personnel, frais assimilés",nan,None,None,D,0.0,0.0,0.0,0.0,1000.000000,459.000000,0.0,0,22790001600352,079090
12999997,757749,79,75.0,200041317,79191,Niort,DEPARTEMENT DES DEUX SEVRES,Département,2021-03-01,Payeur Départemental,nan,nan,Départements/Ets Public Local/Ets Public Santé,09,2020,None,FcIc,true,fa

# On reprend tout 

In [1]:
import duckdb
import pandas as pd 
import polars as pl 
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
df_transco = pd.read_parquet('./ETL_Lignes_Budget_EV/transcodage/parquet/transcodage_2020')
transco = df_transco[['Code', 'Lib_court', 'Libelle', 'Categorie', 'Nomenclature']]
transco1 = transco[['Code', 'Lib_court', 'Categorie', 'Nomenclature']]

df_contnat = transco1[transco1['Categorie'] == 'Nature'][['Code', 'Lib_court', 'Nomenclature']].rename(columns = {
                                                         'Code': 'ContNat', 'Lib_court' : 'ContNat_Lib'})
df_nature = transco1[transco1['Categorie'] == 'Nature_compte'][['Code', 'Lib_court', 'Nomenclature']].rename(columns = {
                                                         'Code': 'Nature', 'Lib_court' : 'Nature_Lib'})
df_fonction = transco1[transco1['Categorie'] == 'Fonction_Ref'][['Code', 'Lib_court', 'Nomenclature']].rename(columns = {
                                                         'Code': 'Fonction', 'Lib_court' : 'Fonction_Lib'})
pl_contnat = pl.from_pandas(df_contnat).lazy()
pl_nature = pl.from_pandas(df_nature).lazy()
pl_fonction = pl.from_pandas(df_fonction).lazy()

In [3]:
d2020 = pl.scan_parquet('./ETL_Lignes_Budget_EV/exploration_viz/db_2020.parquet')\
                .filter(pl.col('Id_Fichier') != '764200',
                        pl.col('NatDec') == '09', 
                        pl.col('OpBudg') == '0')\
                .with_columns(
                        pl.col('DteStr').cast(pl.Date),
                        pl.col('TypOpBudg').replace('nan', None).cast(pl.Int64),
                        pl.col('Exer').cast(pl.Int64),
                        pl.col('ProjetBudget').map_dict({
                                '1' : True, 'true' : True, 
                                '0' : False, 'false' : False,
                                'null': None})
                                .cast(pl.Boolean),
                        pl.col('OpeEquip').map_dict({
                                '1' : True, 'true' : True, 
                                '0' : False, 'false' : False})
                                .cast(pl.Boolean),
                        pl.col('VoteFormelChap').map_dict({
                                '1' : True, 'true' : True, 
                                '0' : False, 'false' : False})
                                .cast(pl.Boolean),
                        pl.col('ArtSpe').map_dict({
                                '1' : True, 'true' : True, 
                                '0' : False, 'false' : False})
                                .cast(pl.Boolean),
                        pl.col('TypProv').cast(pl.Int16),
                        pl.col('BudgPrec').cast(pl.Int16),
                        pl.col('ReprRes').cast(pl.Int16),
                        pl.col('NatFonc').cast(pl.Int16), 
                        pl.col('CodColl').cast(pl.Int16),
                        pl.col('ContFon').replace('nan', None)
                                .cast(pl.Int32))\
                .collect(streaming = True)
#764200 Fichier vide 

/tmp/ipykernel_12190/3328774430.py:9: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The default behavior has changed to keep any values not present in the mapping unchanged. Pass `default=None` to keep existing behavior.
  pl.col('ProjetBudget').map_dict({
/tmp/ipykernel_12190/3328774430.py:14: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The default behavior has changed to keep any values not present in the mapping unchanged. Pass `default=None` to keep existing behavior.
  pl.col('OpeEquip').map_dict({
/tmp/ipykernel_12190/3328774430.py:18: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The default behavior has changed to keep any values not present in the mapping unchanged. Pass `default=None` to keep existing behavior.
  pl.col('VoteFormelChap').map_dict({
/tmp/ipykernel_12190/3328774430.py:22: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The defa

- 764200 Fichier vide 
- IdPost Contient des B
- 

In [4]:
d2020.head(1)

Id_Fichier,IdColl,DteStr,LibellePoste,IdPost,TypOpBudg,OpeCpteTiers,NatCEPL,Departement,NatDec,Exer,ProjetBudget,NatVote,OpeEquip,VoteFormelChap,TypProv,BudgPrec,ReprRes,NatFonc,CodTypBud,LibelleEtabPal,IdEtabPal,LibelleEtab,IdEtab,CodColl,CodInseeColl,CodBud,Nomenclature,Nature,Fonction,LibCpte,ContNat,ArtSpe,ContFon,CodRD,MtBudgPrec,MtRARPrec,MtPropNouv,MtPrev,CredOuv,MtReal,MtRAR3112,OpBudg,MtSup,CaracSup,BudgetHorsRAR,ICNE,ICNEPrec,APVote,ProdChaRat,RARPrec,ProgAutoLib,ProgAutoNum,MtOpeCumul,Brut,Comp,Net,ChapSpe,MtOpeInfo,TypOpe
str,str,date,str,str,i64,str,str,str,str,i64,bool,str,bool,bool,i16,i16,i16,i16,str,str,str,str,str,i16,str,str,str,str,str,str,str,bool,i32,str,f32,f32,f32,f32,f32,f32,f32,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""692101""","""21910540000010…",2021-04-08,"""TP DOURDAN""","""091009""",null,null,"""Commune""","""91""","""09""",2020,false,"""FcIc""",false,false,2,2,2,3,"""P""",null,null,"""BUDGET COMMUNA…","""21910540000010…",100,"""91540""","""00""","""M14-M14_COM_SU…","""6161""","""020""",null,"""011""",false,null,"""D""",null,NaN,55750.0,55750.0,55750.0,55665.441406,NaN,"""0""","""nan""","""{'@Code': 'Sec…",null,"""nan""","""nan""",null,"""nan""",null,null,null,null,null,null,null,null,null,null


df = d2020.lazy().with_columns(
    d2020['IdColl'].cast(pl.Int64), 
    d2020['DteStr'].cast(pl.Date),
    d2020['TypOpBudg'].replace('nan', None).cast(pl.Int64),
    d2020['Exer'].cast(pl.Int64),
    d2020['ProjetBudget'].map_dict({'1' : True, 'true' : True, 
                                '0' : False, 'false' : False,
                                'null': None})
                        .cast(pl.Boolean),
    d2020['OpeEquip'].map_dict({'1' : True, 'true' : True, 
                                '0' : False, 'false' : False})
                        .cast(pl.Boolean),
    d2020['VoteFormelChap'].map_dict({'1' : True, 'true' : True, 
                                '0' : False, 'false' : False})
                        .cast(pl.Boolean),
    d2020['ArtSpe'].map_dict({'1' : True, 'true' : True, 
                                '0' : False, 'false' : False})
                        .cast(pl.Boolean),
    d2020['TypProv'].cast(pl.Int16),
    d2020['BudgPrec'].cast(pl.Int16),
    d2020['ReprRes'].cast(pl.Int16),
    d2020['NatFonc'].cast(pl.Int16), 
    d2020['CodColl'].cast(pl.Int16),
    d2020['ContFon'].replace('nan', None).cast(pl.Int32),
).collect(streaming = True)

In [5]:
source_siren = pl.scan_parquet('../projets/data/infos_siren_siret_cog_collectivites.parquet').lazy()\
                     .select(['DEP', 'REG', 'CODGEO', 'LIBGEO', 'denominationUniteLegale', 'libelleCategorieJuridique', 'siret'])\
                     .collect(streaming = True)

In [6]:
df1 = d2020.lazy().join(pl_contnat.lazy(), on = ['Nomenclature','ContNat'], how = 'left')\
                .join(pl_nature.lazy(), on = ['Nomenclature','Nature'], how = 'left')\
                .join(pl_fonction.lazy(), on = ['Nomenclature','Fonction'], how = 'left')\
                .join(source_siren.lazy(), left_on = 'IdColl', right_on = 'siret', how = 'left', suffix = '')\
                .collect(streaming = True)



In [7]:
df1.shape

(6204707, 69)

In [10]:
df1 = df1[['Id_Fichier', 'IdColl', 'Exer', 'Nomenclature','DEP', 'REG', 'CODGEO', 
        'LIBGEO','DteStr', 'LibellePoste', 'denominationUniteLegale',
        'libelleCategorieJuridique','IdPost', 'TypOpBudg', 'OpBudg', 'OpeCpteTiers',
        'NatCEPL', 'NatDec', 'ProjetBudget', 'NatVote', 'OpeEquip', 'VoteFormelChap', 
        'TypProv', 'BudgPrec', 'ReprRes', 'NatFonc', 'CodTypBud', 'LibelleEtabPal',
        'IdEtabPal', 'LibelleEtab',	'IdEtab', 'CodColl', 'CodInseeColl', 'CodBud',
        'TypOpe', 'LibCpte', 'CodRD', 'Nature', 'Nature_Lib', 'ContNat', 'ContNat_Lib', 
        'Fonction', 'Fonction_Lib', 'ContFon', 'ArtSpe', 'MtBudgPrec', 'RARPrec',
        "MtRARPrec", 'MtPropNouv', 'MtPrev', 'CredOuv', 'MtReal', 'MtRAR3112']]
df1.head(1)

Id_Fichier,IdColl,Exer,Nomenclature,DEP,REG,CODGEO,LIBGEO,DteStr,LibellePoste,denominationUniteLegale,libelleCategorieJuridique,IdPost,TypOpBudg,OpBudg,OpeCpteTiers,NatCEPL,NatDec,ProjetBudget,NatVote,OpeEquip,VoteFormelChap,TypProv,BudgPrec,ReprRes,NatFonc,CodTypBud,LibelleEtabPal,IdEtabPal,LibelleEtab,IdEtab,CodColl,CodInseeColl,CodBud,TypOpe,LibCpte,CodRD,Nature,Nature_Lib,ContNat,ContNat_Lib,Fonction,Fonction_Lib,ContFon,ArtSpe,MtBudgPrec,RARPrec,MtRARPrec,MtPropNouv,MtPrev,CredOuv,MtReal,MtRAR3112
str,str,i64,str,str,i64,str,str,date,str,str,str,str,i64,str,str,str,str,bool,str,bool,bool,i16,i16,i16,i16,str,str,str,str,str,i16,str,str,str,str,str,str,str,str,str,str,str,i32,bool,f32,str,f32,f32,f32,f32,f32,f32
"""692101""","""21910540000010…",2020,"""M14-M14_COM_SU…","""91""",11,"""91540""","""Saint-Chéron""",2021-04-08,"""TP DOURDAN""","""COMMUNE DE SAI…","""Commune et com…","""091009""",null,"""0""",null,"""Commune""","""09""",false,"""FcIc""",false,false,2,2,2,3,"""P""",null,null,"""BUDGET COMMUNA…","""21910540000010…",100,"""91540""","""00""",null,null,"""D""","""6161""","""Multirisques""","""011""","""Charges à cara…","""020""","""Administrat°gé…",null,false,null,null,NaN,55750.0,55750.0,55750.0,55665.441406,NaN


In [8]:
df1.estimated_size('mb')

4764.900776863098

In [10]:
df1.lazy().sink_parquet('./hub_coll_2020_join.parquet')

In [10]:
df_verif = pl.scan_parquet('./hub_coll_2020_join.parquet').filter(pl.col('Fonction') == '91821').collect()


In [11]:
df_verif

Id_Fichier,IdColl,DteStr,LibellePoste,IdPost,TypOpBudg,OpeCpteTiers,NatCEPL,Departement,NatDec,Exer,ProjetBudget,NatVote,OpeEquip,VoteFormelChap,TypProv,BudgPrec,ReprRes,NatFonc,CodTypBud,LibelleEtabPal,IdEtabPal,LibelleEtab,IdEtab,CodColl,CodInseeColl,CodBud,Nomenclature,Nature,Fonction,LibCpte,ContNat,ArtSpe,ContFon,CodRD,MtBudgPrec,MtRARPrec,MtPropNouv,MtPrev,CredOuv,MtReal,MtRAR3112,OpBudg,MtSup,CaracSup,BudgetHorsRAR,ICNE,ICNEPrec,APVote,ProdChaRat,RARPrec,ProgAutoLib,ProgAutoNum,MtOpeCumul,Brut,Comp,Net,ChapSpe,MtOpeInfo,TypOpe,ContNat_Lib,Nature_Lib,Fonction_Lib,DEP,REG,CODGEO,LIBGEO,denominationUniteLegale,libelleCategorieJuridique
str,str,date,str,str,i64,str,str,str,str,i64,bool,str,bool,bool,i16,i16,i16,i16,str,str,str,str,str,i16,str,str,str,str,str,str,str,bool,i32,str,f32,f32,f32,f32,f32,f32,f32,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str
"""777394""","""22870851700989…",2021-02-18,"""PAYEUR DEPARTE…","""087090""",null,null,"""CONSEIL DEPART…",null,"""09""",2020,false,"""FcIc""",false,false,1,1,1,2,"""P""",null,null,"""BUDGET PRINCIP…","""22870851700989…",16,null,"""00""","""M52-M52""","""204141""","""91821""",null,null,false,918,"""R""",null,NaN,null,0.0,NaN,21903.75,NaN,"""0""","""{'@Code': 'Bud…","""nan""","""0.00""","""nan""","""nan""",null,"""nan""",null,"""nan""","""nan""",null,null,null,null,"""nan""",null,null,null,"""Subv.Cne : Bie…",null,"""87""",75,"""87085""","""Limoges""","""DEPARTEMENT DE…","""Département"""
"""767910""","""22620001200012…",2021-05-25,"""Paierie départ…","""062090""",null,"""nan""","""Département""",null,"""09""",2020,null,"""FcIc""",false,false,1,1,1,2,"""P""",null,"""22620001200012…","""Budget princip…","""22620001200012…",5,null,"""00""","""M52-M52""","""204142""","""91821""","""nan""",null,false,918,"""D""",0.0,0.0,0.0,0.0,25000.0,25000.0,0.0,"""0""","""[{'@Code': 'Pr…","""{'@Code': 'Pro…","""25000.00""","""nan""","""nan""",null,"""0.00""",null,"""nan""","""AP""","""nan""",null,null,null,"""nan""",null,null,null,"""Subv.Cne : Bât…",null,"""62""",32,"""62041""","""Arras""","""DEPARTEMENT DU…","""Département"""
"""761859""","""22620001200012…",2021-05-25,"""Paierie départ…","""062090""",null,"""nan""","""Département""",null,"""09""",2020,null,"""FcIc""",false,false,1,1,1,2,"""P""",null,"""22620001200012…","""Budget princip…","""22620001200012…",5,null,"""00""","""M52-M52""","""204142""","""91821""","""nan""",null,false,918,"""D""",0.0,0.0,0.0,0.0,25000.0,25000.0,0.0,"""0""","""[{'@Code': 'Pr…","""{'@Code': 'Pro…","""25000.00""","""nan""","""nan""",null,"""0.00""",null,"""nan""","""AP""","""nan""",null,null,null,"""nan""",null,null,null,"""Subv.Cne : Bât…",null,"""62""",32,"""62041""","""Arras""","""DEPARTEMENT DU…","""Département"""
"""749354""","""22060001900016…",2021-04-20,"""PAIERIE DEPART…","""006090""",null,"""nan""","""CONSEIL DEPART…",null,"""09""",2020,false,"""FcIc""",false,false,null,1,3,2,"""P""",null,"""22060001900016…","""Conseil Départ…","""22060001900016…",6,null,"""00""","""M52-M52""","""204142""","""91821""","""nan""",null,false,918,"""D""",null,null,null,0.0,1e6,0.0,null,"""0""","""{'@Code': 'Bud…","""[{'@Code': 'Pr…","""1000000.00""","""nan""","""nan""",null,"""nan""",null,"""CONTRAT DE PLA…","""D D51CONTRAT 2…","""nan""",null,null,null,"""nan""",null,null,null,"""Subv.Cne : Bât…",null,"""06""",93,"""06088""","""Nice""","""DEPARTEMENT DE…","""Département"""
"""749354""","""22060001900016…",2021-04-20,"""PAIERIE DEPART…","""006090""",null,"""nan""","""CONSEIL DEPART…",null,"""09""",2020,false,"""FcIc""",false,false,null,1,3,2,"""P""",null,"""22060001900016…","""Conseil Départ…","""22060001900016…",6,null,"""00""","""M52-M52""","""204142""","""91821""","""nan""",null,false,918,"""D""",null,null,null,0.0,2.94e6,0.0,null,"""0""","""{'@Code': 'Bud…","""[{'@Code': 'Pr…","""2940000.00""","""nan""","""nan""",null,"""nan""",null,"""CONTRAT DE PLA…","""D D51CONTRAT 2…","""nan""",null,null,null,"""nan""",null

In [3]:
df_categ = pd.read_parquet('../../data/infos_siren_siret_cog_collectivites.parquet')



,siren,statutDiffusionUniteLegale,unitePurgeeUniteLegale,dateCreationUniteLegale,sigleUniteLegale,sexeUniteLegale,prenom1UniteLegale,prenom2UniteLegale,prenom3UniteLegale,prenom4UniteLegale,...,CV,ZE2020,UU2020,TUU2017,TDUU2017,AAV2020,TAAV2017,TDAAV2017,CATEAAV2020,BV2022
0,130001589,O,None,2005-12-25,CDAD,None,None,None,None,None,...,0199,8405,01501,5,51,080,2,24,11,01053
1,130001589,O,None,2005-12-25,CDAD,None,None,None,None,None,...,0199,8405,01501,5,51,080,2,24,11,01053
2,130001605,O,None,2005-12-25,CDAD,None,None,None,None,None,...,1598,8403,15401,4,43,123,2,22,11,15014


In [4]:
pd.set_option('display.max_columns', 500)
df_categ.head(3)

,siren,statutDiffusionUniteLegale,unitePurgeeUniteLegale,dateCreationUniteLegale,sigleUniteLegale,sexeUniteLegale,prenom1UniteLegale,prenom2UniteLegale,prenom3UniteLegale,prenom4UniteLegale,prenomUsuelUniteLegale,pseudonymeUniteLegale,identifiantAssociationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,dateDernierTraitementUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,dateDebut,etatAdministratifUniteLegale,nomUniteLegale,nomUsageUniteLegale,denominationUniteLegale,denominationUsuelle1UniteLegale,denominationUsuelle2UniteLegale,denominationUsuelle3UniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,nomenclatureActivitePrincipaleUniteLegale,nicSiegeUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,caractereEmployeurUniteLegale,categorieJuridiqueUniteLegale:1,libelleCategorieJuridique,siren:1,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,nombrePeriodesEtablissement,complementAdresseEtablissement,numeroVoieEtablissement,indiceRepetitionEtablissement,typeVoieEtablissement,libelleVoieEtablissement,codePostalEtablissement,libelleCommuneEtablissement,libelleCommuneEtrangerEtablissement,distributionSpecialeEtablissement,codeCommuneEtablissement,codeCedexEtablissement,libelleCedexEtablissement,codePaysEtrangerEtablissement,libellePaysEtrangerEtablissement,complementAdresse2Etablissement,numeroVoie2Etablissement,indiceRepetition2Etablissement,typeVoie2Etablissement,libelleVoie2Etablissement,codePostal2Etablissement,libelleCommune2Etablissement,libelleCommuneEtranger2Etablissement,distributionSpeciale2Etablissement,codeCommune2Etablissement,codeCedex2Etablissement,libelleCedex2Etablissement,codePaysEtranger2Etablissement,libellePaysEtranger2Etablissement,dateDebut:1,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement,CODGEO,LIBGEO,DEP,REG,EPCI,NATURE_EPCI,ARR,CV,ZE2020,UU2020,TUU2017,TDUU2017,AAV2020,TAAV2017,TDAAV2017,CATEAAV2020,BV2022
0,130001589,O,None,2005-12-25,CDAD,None,None,None,None,None,None,None,None,02,2021.0,2023-11-30T10:17:02,4,PME,2021.0,2016-04-01,A,None,None,CONSEIL DEP DE L'ACCES AU DROIT,None,None,None,7410,84.23Z,NAFRev2,22,None,None,O,7410,Groupement d'intérêt public (GIP),130001589,14,13000158900014,O,2005-12-25,NN,NaN,None,2019-11-14T14:00:12,False,4,TRIBUNAL DE GRANDE INSTANCE,4,None,RUE,DU PALAIS,01000,BOURG-EN-BRESSE,None,BP 306,01053,01011,BOURG EN BRESSE CEDEX,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2016-04-01,F,None,None,None,None,84.23Z,NAFRev2,O,01053,Bourg-en-Bresse,01,84,200071751,CA,012,0199,8405,01501,5,51,080,2,24,11,01053
1,130001589,O,None,2005-12-25,CDAD,None,None,None,None,None,None,None,None,02,2021.0,2023-11-30T10:17:02,4,PME,2021.0,2016-04-01,A,None,None,CONSEIL DEP DE L'ACCES AU DROIT,None,None,None,7410,84.23Z,NAFRev2,22,None,None,O,7410,Groupement d'intérêt public (GIP),130001589,22,13000158900022,O,2016-04-01,02,2021.0,None,2023-11-30T10:17:02,True,1,TRIBUNAL DE GRANDE INSTANCE,32,None,AV,ALSACE LORRAINE,01000,BOURG-EN-BRESSE,None,None,01053,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2016-04-01,A,None,None,None,None,84.23Z,NAFRev2,O,01053,Bourg-en-Bresse,01,84,200071751,CA,012,0199,8405,01501,5,51,080,2,24,11,01053
2,130001605,O,None,2005-12-25,CDAD,None,None,None,None,None,None,None,None,01,2021.0,2023-11-30T10:17:02,3,PME,2021.0,2008-01-01,A,None,None,CONSEIL DEP DE L'ACCES AU DROIT,None,None,None,7410,84.23Z,NAFRev2,18,None,None,O,7410,Groupement d'intérêt public (GIP),130001605,18,13000160500018,O,2005-12-25,01,2021.0,None,2023-11-30T10:17:02,Tru